In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import re

In [2]:
contents, sympathy, antipathy, periods = [], [], [], []
last_no = 20
for no in tqdm(range(1, last_no+1)):
    url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json'

    params_list = 'ticket=comic&templateId=webtoon&pool=cbox3&_cv=20221018144728&_callback=jQuery112409798401549196138_1667260488026&lang=ko&country=KR&objectId=783054_13&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=10&_=1667260488028'.split('&')
    params = {
        'ticket':'comic',
        'templateId':'webtoon',
        'pool':'cbox3',
        '_cv': 20221018144728,
        '_callback':'jQuery112409798401549196138_1667260488026',
        'lang':'ko',
        'country':'KR',
        'objectId':'783054_{}'.format(no),
        'pageSize':15,
        'indexSize':10,
        'listType':'OBJECT',
        'pageType':'default',
        'page':1,
        'initialize':'true',
        'useAltSort':'true',
        'replyPageSize':10
    }

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
            'referer' : 'https://comic.naver.com/comment/comment?titleId=783054&no=13'}
    resp = requests.get(url, params=params, headers=headers)

    result = resp.text[42:-2]
    result = json.loads(result)
    commentlists = result['result']['commentList']

    for commentlist in commentlists:
        contents.append(commentlist['contents'])
        sympathy.append(commentlist['sympathyCount'])
        antipathy.append(commentlist['antipathyCount'])
        periods.append(no)

commentList = {
    "댓글" : contents,
    "공감" : sympathy,
    "비공감" : antipathy,
    "회차" : periods
}

100%|██████████| 20/20 [00:06<00:00,  2.88it/s]


In [24]:
re.findall('callback(.+)', resp.text)

error: ignored

In [ ]:
contents, sympathy, antipathy, periods = [], [], [], []
last_no = 20
for no in tqdm(range(1, last_no+1)):
    url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json'

    params_list = 'ticket=comic&templateId=webtoon&pool=cbox3&_cv=20221018144728&_callback=jQuery112409798401549196138_1667260488026&lang=ko&country=KR&objectId=783054_13&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=10&_=1667260488028'.split('&')
    params = {
        'ticket':'comic',
        'templateId':'webtoon',
        'pool':'cbox3',
        '_cv': 20221018144728,
        '_callback':'jQuery112409798401549196138_1667260488026',
        'lang':'ko',
        'country':'KR',
        'objectId':'783054_{}'.format(no),
        'pageSize':15,
        'indexSize':10,
        'listType':'OBJECT',
        'pageType':'default',
        'page':1,
        'initialize':'true',
        'useAltSort':'true',
        'replyPageSize':10
    }

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
            'referer' : 'https://comic.naver.com/comment/comment?titleId=783054&no=13'}
    resp = requests.get(url, params=params, headers=headers)

    result = resp.text[42:-2]
    result = json.loads(result)
    commentlists = result['result']['commentList']

    for commentlist in commentlists:
        contents.append(commentlist['contents'])
        sympathy.append(commentlist['sympathyCount'])
        antipathy.append(commentlist['antipathyCount'])
        periods.append(no)

commentList = {
    "댓글" : contents,
    "공감" : sympathy,
    "비공감" : antipathy,
    "회차" : periods
}

commentList = pd.DataFrame(commentList)
commentList.to_csv('221101 쇼미더럭키짱 전체베댓.csv')

100%|██████████| 20/20 [00:11<00:00,  1.75it/s]


전체 웹툰 최신화 베스트댓글 가져오기

- 완결웹툰 포함

In [ ]:
url1 = 'https://comic.naver.com/webtoon/weekday' # 현재 연재중인 전체웹툰
url2 = 'https://comic.naver.com/webtoon/finish'  # 완결된 웹툰

resp1 = requests.get(url1)
resp2 = requests.get(url2)
resp2

<Response [200]>

In [ ]:
soup1 = BeautifulSoup(resp1.content, 'lxml')
soup2 = BeautifulSoup(resp2.content, 'lxml')
tit_tags1 = soup1.select('div.col li a.title')
tit_tags2 = soup2.select('ul.img_list li dl dt a')
tit_tags = tit_tags1 + tit_tags2

In [ ]:
titles, urls, ids, nos = [], [], [], []

for tit_tag in tqdm(tit_tags):
    titles.append(tit_tag['title'])
    urls.append('https://comic.naver.com'+tit_tag['href'])
    
    id = tit_tag['href'].split('=')[1].rsplit('&')[0]
    ids.append(id)


for url in tqdm(urls):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    latest_num = soup.select('td.title a')[0]['href'].split('&')[1].replace('no=','')
    latest_num = int(latest_num)
    nos.append(latest_num)


100%|██████████| 1920/1920 [11:34<00:00,  2.77it/s]


In [ ]:
toon_info = {}

for i in tqdm(range(len(titles))):
    toon_info[titles[i]] = {'title_id' : ids[i], 'no' : int(nos[i]), 'url' : urls[i]}

100%|██████████| 1920/1920 [00:00<00:00, 543831.96it/s]


In [ ]:
myToon = input('어떤 웹툰의 댓글을 크롤링할까요? : ')

어떤 웹툰의 댓글을 크롤링할까요? : 쇼미더럭키짱!


In [ ]:
print(toon_info[myToon]['no'])
print(toon_info[myToon]['title_id'])

my_new_no = toon_info[myToon]['no']
my_title_id = toon_info[myToon]['title_id']

20
783054


In [ ]:
def func(a, b):
    result = a + b
    return result

In [ ]:
def toon_review_scraping(last_no, title_id):
    
    reviews = {
        '댓글' : [],
        '공감수' : [],
        '비공감수' : [],
        '회차' : []
    }

    for no in tqdm(range(1, latest_num+1)):
        url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'


        params ={
                'ticket' : 'comic',  
                'templateId' : 'webtoon',  
                'pool' : 'cbox3',  
                '_callback' : 'jQuery11240590927015056502_1658103118387', # jQuery112409091552467251651_1615854232166 -> ''  
                'lang' : 'ko',  
                'country' : 'KR',  
                'objectId' : '{}_{}'.format(title_id, no), # 웹툰의 고유 아이디_해당화 인덱스  
                'pageSize' : 15,  
                'indexSize' : 10,  
                'listType' : 'OBJECT',  
                'pageType' : 'default',  
                'page' : 1, # 전체댓글의 페이지 번호 
                'refresh' : 'true',  
                }

        headers = {
                'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
        }

        resp = requests.get(url, params=params, headers=headers)
        
        jsn = resp.text[41:-2]

        dict_resp = json.loads(jsn)

        commentList = dict_resp['result']['commentList']

        for comment in commentList:
            reviews['댓글'].append(comment['contents'])
            reviews['공감수'].append(comment['sympathyCount'])
            reviews['비공감수'].append(comment['antipathyCount'])
            reviews['회차'].append(no)
    return reviews

In [ ]:
mytoon = input('웹툰제목을 입력해주세요:')

웹툰제목을 입력해주세요:쇼미더럭키짱!


In [ ]:
title_id = toon_info[myToon]['title_id']
last_no = toon_info[myToon]['no']

reviews = toon_review_scraping(latest_num, title_id)

100%|██████████| 31/31 [00:13<00:00,  2.25it/s]


In [ ]:
df_reviews = pd.DataFrame(reviews)
df_reviews

,댓글,공감수,비공감수,회차
0,주5회ㅋㅋㅋ 래전드네ㅋㄱㅋ,18329,85,1
1,김성모 + 박서준ㅋㅋㅋㅋ 어떤 의미로든 레전드의 조합ㅋㅋㅋㅋㅋㅋ,12407,371,1
2,잠만 부산에 왜 남산타워가 있고 서울에 갈매기가 돌아다니지?,12364,103,1
3,성모 형님 오래간만에 오셨네요. 오래간만에 왔으니 별점 1점 드립니다.,6493,326,1
4,느슨해진 웹툰계에 긴장을 주기 위해 떨어진 핵폭탄,6280,44,1
...,...,...,...,...
400,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ압축은 뭔데 ㅋㅋㅋㅋ,3757,50,31
401,ㅋㅋㅋㅋㅋ 개그만화로 최고다,2467,36,31
402,???: 훠훠훠 싸움과 돼통령이라... 재밌는 이슈입니돠..\n당장 이거 만든 작가...,2445,597,31
403,ㅁㅊ...내가 이걸 쿠키굽고있는것도 자존심 상하는데 ㄹㅇ 이번화 레전드네 개웃겨,1442,63,31
